# Карьерный навигатор HH
# Описание проекта

Проект направлен на создание карьерного навигатора, который поможет пользователям находить вакансии на hh.ru, анализировать данные по ним и получать рекомендации на основе введенных ключевых навыков или текста резюме. Это включает:

Парсинг вакансий с hh.ru по ряду условий.
Агрегацию данных для анализа различий между различными наборами вакансий.
Создание рекомендательной системы для поиска подходящих вакансий на основе пользовательских навыков.

Парсинг вакансий с hh.ru:

Сбор информации по вакансиям с hh.ru через API.
Обработка данных для извлечения ключевой информации о вакансиях.
Вычисление средней зарплаты и анализ частоты биграмм в названиях вакансий.
Агрегация данных:
Сравнение двух наборов данных с вакансиями для выявления различий.

Рекомендательная система:

Создание системы, которая на основе пользовательских навыков рекомендует подходящие вакансии.

# Навыки и инструменты
Библиотеки Python:
requests для выполнения HTTP-запросов к API hh.ru.
pandas для обработки и анализа данных.
collections.Counter для частотного анализа биграмм.
sklearn.feature_extraction.text.TfidfVectorizer и sklearn.metrics.pairwise.cosine_similarity для создания рекомендательной системы.
API hh.ru: Получение данных о вакансиях.
Google Sheets и gdown: Загрузка и работа с данными, предоставленными в формате Excel.

# Вывод:
Проект успешно демонстрирует автоматизированный сбор и анализ данных о вакансиях с hh.ru, а также создание прототипа рекомендательной системы для поиска подходящих вакансий на основе навыков пользователя. Реализованный скрипт собирает информацию о вакансиях по четырем специальностям и сохраняет её в удобном для анализа формате. Выполнен анализ средней зарплаты по каждой специальности и выявлены 10 самых частых биграмм в названиях вакансий. Сравнены два набора данных с вакансиями интернет-маркетологов, что позволило выявить вакансии, присутствующие в одном наборе, но отсутствующие в другом. Разработан прототип рекомендательной системы, который на основе введенных ключевых навыков пользователя рекомендует 10 наиболее подходящих вакансий. Проект может служить основой для более сложных систем анализа и рекомендаций вакансий, упрощая процесс поиска работы для соискателей.

#1 Создание парсера вакансий по ряду условий:
A) Необходимо создать код по закачке вакансий с hh.ru через API. Итоговый датафрейм должен содержать основную информацию по вакансиям:
- Название
- Уровень опыта
- Ключевые навыки
- Профессиональную роль
- Ссылку на вакансию
- Город
- Зарплату

B) Закачать вакансии по специальностям:
- Разработчик Java
- SMM
- Интернет-маркетолог
- Веб-дизайнер

C) Показать:
- Среднюю зарплату по каждой из специальностей
- 10 самых частых би-грамм для каждой из специальностей



In [39]:
# Импорт библиотек
import requests
import pandas as pd
from collections import Counter

In [40]:
# Функция для загрузки вакансий с помощью API HH.ru
def fetch_vacancies(keyword, pages=10):
    # URL API HH.ru для загрузки вакансий
    url = "https://api.hh.ru/vacancies"
    # Список для хранения всех вакансий
    all_vacancies = []
    # Перебор страниц для загрузки
    for page in range(0, pages):
        # Параметры запроса: ключевое слово, регион (Москва), количество вакансий на странице, номер страницы
        params = {
            "text": keyword,
            "area": 1,  # Код региона (Москва)
            "per_page": 100,  # Количество вакансий на странице
            "page": page
        }
        # Выполнение GET-запроса к API HH.ru
        response = requests.get(url, params=params)
        # Преобразование ответа в формат JSON
        data = response.json()
        # Добавление загруженных вакансий к общему списку
        all_vacancies.extend(data['items'])
    # Возвращение списка всех загруженных вакансий
    return all_vacancies

In [41]:
# Функция для извлечения информации о вакансии
def extract_info(vacancy):
    # Извлечение основной информации о вакансии: название, опыт, ключевые навыки, роль, ссылка, город, зарплата
    title = vacancy['name']
    experience = vacancy['experience']['name'] if 'experience' in vacancy else None
    key_skills = ', '.join(skill['name'] for skill in vacancy.get('key_skills', []))
    professional_role = vacancy['specializations'][0]['profarea_name']
    link = vacancy['alternate_url']
    city = vacancy['area']['name']
    salary = vacancy['salary'] if vacancy['salary'] else {'from': None, 'to': None}
    return {
        'title': title,
        'experience': experience,
        'key_skills': key_skills,
        'professional_role': professional_role,
        'link': link,
        'city': city,
        'salary_from': salary['from'],
        'salary_to': salary['to']
    }

# Функция для вычисления средней зарплаты
def calculate_avg_salary(vacancies):
    salaries = []
    # Перебор вакансий
    for vacancy in vacancies:
        # Проверка наличия информации о зарплате
        if vacancy['salary']:
            # Добавление минимальной и максимальной зарплаты в список
            if vacancy['salary']['from']:
                salaries.append(vacancy['salary']['from'])
            if vacancy['salary']['to']:
                salaries.append(vacancy['salary']['to'])
    # Вычисление средней зарплаты
    if salaries:
        return sum(salaries) / len(salaries)
    else:
        return None

In [42]:
# Функция для получения топ-10 биграмм
def get_top_bigrams(vacancies):
    bigrams = []
    # Перебор вакансий
    for vacancy in vacancies:
        # Преобразование названия вакансии в нижний регистр и разделение на слова
        title = vacancy['name'].lower().split()
        # Формирование биграмм и добавление их в список
        bigrams.extend(zip(title, title[1:]))
    # Подсчет частоты встречаемости биграмм
    counter = Counter(bigrams)
    # Возвращение топ-10 самых часто встречающихся биграмм
    return counter.most_common(10)

In [43]:
# Список специализаций для поиска
specializations = ['разработчик Java', 'SMM', 'интернет-маркетолог', 'веб-дизайнер']

In [44]:
# Список для сохранения результатов по каждой специализации
data = []

In [45]:
# Получение данных по каждой специализации
for specialization in specializations:
    # Загрузка вакансий по текущей специализации (1000 вакансий)
    vacancies = fetch_vacancies(specialization, pages=10)
    # Вычисление средней зарплаты
    avg_salary = calculate_avg_salary(vacancies)
    # Получение топ-10 биграмм
    top_bigrams = get_top_bigrams(vacancies)
    # Сохранение результатов для текущей специализации
    data.append({
        'specialization': specialization,
        'average_salary': avg_salary,
        'top_bigrams': top_bigrams
    })

In [46]:
# Вывод результатов
for item in data:
    print(f"Специализация: {item['specialization']}")
    print(f"Средняя зарплата: {item['average_salary']} рублей" if item['average_salary'] else "Средняя зарплата не доступна")
    print("Топ 10 самых частых биграмм:")
    for bigram, count in item['top_bigrams']:
        print(f"{bigram[0]} {bigram[1]} - {count} раз")
    print("\n")


Специализация: разработчик Java
Средняя зарплата: 206484.24657534246 рублей
Топ 10 самых частых биграмм:
java developer - 110 раз
java разработчик - 91 раз
разработчик java - 40 раз
системный аналитик - 36 раз
senior java - 33 раз
qa engineer - 22 раз
backend developer - 16 раз
android developer - 14 раз
middle java - 13 раз
team lead - 13 раз


Специализация: SMM
Средняя зарплата: 93499.38860103628 рублей
Топ 10 самых частых биграмм:
менеджер по - 88 раз
по маркетингу - 37 раз
руководитель отдела - 35 раз
по работе - 30 раз
работе с - 30 раз
отдела маркетинга - 23 раз
специалист по - 22 раз
smm менеджер - 22 раз
директор по - 20 раз
ассистент руководителя - 17 раз


Специализация: интернет-маркетолог
Средняя зарплата: 119188.69017094017 рублей
Топ 10 самых частых биграмм:
менеджер по - 34 раз
интернет-маркетолог / - 20 раз
/ интернет-маркетолог - 12 раз
специалист по - 11 раз
по работе - 11 раз
работе с - 11 раз
маркетолог / - 10 раз
digital маркетолог - 9 раз
/ digital - 9 раз
digita

# 2 Пример агрегации данных

Возьмём два файла с вакансиями, прилагающиеся к тексту задания: **internet_marketologs_part_1.xlsx** и **internet_marketologs_part_2.xlsx**. Создадим датафрейм с вакансиями, вошедшими в первую закачку, но не вошедшими во вторую.

https://docs.google.com/spreadsheets/d/1R7lMQnvEfwpbzCpIKxmODaNx_FpWzmWz/edit?usp=sharing&ouid=108293426721785695542&rtpof=true&sd=true

In [47]:
# Загрузка первого файла
! gdown --id 1R7lMQnvEfwpbzCpIKxmODaNx_FpWzmWz

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1R7lMQnvEfwpbzCpIKxmODaNx_FpWzmWz
To: /content/internet_marketologs_part_1.xlsx
100% 963k/963k [00:00<00:00, 108MB/s]


https://docs.google.com/spreadsheets/d/1zzgUmWjNS8sr4JnUm3GOakN2xkEvcpQU/edit?usp=sharing&ouid=108293426721785695542&rtpof=true&sd=true

In [48]:
# Загрузка данных из второго файла
! gdown --id 1zzgUmWjNS8sr4JnUm3GOakN2xkEvcpQU

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zzgUmWjNS8sr4JnUm3GOakN2xkEvcpQU
To: /content/internet_marketologs_part_2.xlsx
100% 983k/983k [00:00<00:00, 110MB/s]


In [49]:
# Импорт библиотеки
import pandas as pd

In [50]:
# Загрузка данных из первого файла
df_part_1 = pd.read_excel("/content/internet_marketologs_part_1.xlsx")
# Загрузка данных из второго файла
df_part_2 = pd.read_excel("/content/internet_marketologs_part_2.xlsx")

In [51]:
# Создание датафрейма с вакансиями, вошедшими в первую закачку, но не вошедшими во вторую
df_difference = pd.concat([df_part_1, df_part_2]).drop_duplicates(keep=False)

In [52]:
# Вывод результата
print("Вакансии, вошедшие в первую закачку, но не вошедшие во вторую:")
print(df_difference)

Вакансии, вошедшие в первую закачку, но не вошедшие во вторую:
                                                   name  \
0                         Помощник интернет-маркетолога   
2                                   Интернет-маркетолог   
3                                   Интернет-маркетолог   
5     Помощник руководителя (маркетолога)/Ученик/Инт...   
6                            Junior интернет-маркетолог   
...                                                 ...   
1049        Интернет-маркетолог/ Менеджер по маркетингу   
1054  Менеджер по интернет-маркетингу (Digital Marke...   
1057                     Интернет-маркетолог (удаленно)   
1059                                Интернет-маркетолог   
1061                                Интернет-маркетолог   

                                                 skills    experiences  \
0     интернет-реклама, seo, яндекс.директ, таргетин...      Нет опыта   
2     интернет-реклама, интернет маркетинг, планиров...      Нет опыта   
3     

#3 Создание рекомендательной системы

Построим прототип рекомендательной системы на основе данных, полученных из закачек в задании 1. Система должна выдавать нужную вакансию при запросе. Например, мы даем ей ключевые навыки или текст резюме интернет маркетолога, на что система выдает нам 10 подходящих вакансий из закачанной в задании 1 базе.

In [53]:
# Импорт бибилотек
import requests
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
# Функция для загрузки вакансий с помощью API HH.ru
def fetch_vacancies(keyword, pages=10):
    url = "https://api.hh.ru/vacancies"
    all_vacancies = []
    for page in range(0, pages):
        params = {
            "text": keyword,
            "area": 1,  # Код региона (Москва)
            "per_page": 100,  # Количество вакансий на странице
            "page": page
        }
        response = requests.get(url, params=params)
        data = response.json()
        all_vacancies.extend(data['items'])
    return all_vacancies

In [55]:
# Функция для извлечения информации о вакансии
def extract_info(vacancy):
    title = vacancy['name']
    experience = vacancy['experience']['name'] if 'experience' in vacancy else None
    key_skills = ', '.join(skill['name'] for skill in vacancy.get('key_skills', []))
    professional_role = vacancy['specializations'][0]['profarea_name']
    link = vacancy['alternate_url']
    city = vacancy['area']['name']
    salary = vacancy['salary'] if vacancy['salary'] else {'from': None, 'to': None}
    return {
        'title': title,
        'experience': experience,
        'key_skills': key_skills,
        'professional_role': professional_role,
        'link': link,
        'city': city,
        'salary_from': salary['from'],
        'salary_to': salary['to']
    }

In [56]:
# Функция для вычисления средней зарплаты
def calculate_avg_salary(vacancies):
    salaries = []
    for vacancy in vacancies:
        if vacancy['salary']:
            if vacancy['salary']['from']:
                salaries.append(vacancy['salary']['from'])
            if vacancy['salary']['to']:
                salaries.append(vacancy['salary']['to'])
    if salaries:
        return sum(salaries) / len(salaries)
    else:
        return None

In [57]:
# Функция для получения топ-10 биграмм
def get_top_bigrams(vacancies):
    bigrams = []
    for vacancy in vacancies:
        title = vacancy['name'].lower().split()
        bigrams.extend(zip(title, title[1:]))
    counter = Counter(bigrams)
    return counter.most_common(10)

# Список специализаций для поиска
specializations = ['разработчик Java', 'SMM', 'интернет-маркетолог', 'веб-дизайнер']
all_vacancies = []

# Получение данных по каждой специализации
for specialization in specializations:
    vacancies = fetch_vacancies(specialization, pages=10)
    all_vacancies.extend(vacancies)

In [58]:
# Функция для рекомендации вакансий на основе ключевых навыков
def recommend_vacancies(user_skills, all_vacancies):
    vacancy_skills = []

    # Извлекаем ключевые навыки из вакансий, если они есть
    for vacancy in all_vacancies:
        if 'key_skills' in vacancy:
            vacancy_skills.append(", ".join(vacancy['key_skills']))
        else:
            vacancy_skills.append("")  # Если ключевых навыков нет, добавляем пустую строку

    # Объединяем ключевые навыки пользователя и навыки из вакансий
    corpus = [user_skills] + vacancy_skills

    # Создаем TF-IDF векторизатор и преобразуем корпус текста в матрицу TF-IDF признаков
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Вычисляем сходство между запросом пользователя и каждой вакансией
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

    # Получаем индексы вакансий, отсортированных по убыванию сходства
    related_indices = cosine_similarities.argsort()[::-1]

    # Возвращаем топ-10 наиболее подходящих вакансий
    top_vacancies = []
    for i in range(10):
        index = related_indices[i]
        top_vacancies.append(all_vacancies[index])
    return top_vacancies

In [59]:
# Пример использования
user_skills = "аналитическое мышление, работа в команде, анализ данных"
recommended_vacancies = recommend_vacancies(user_skills, all_vacancies)

In [60]:
# Вывод рекомендованных вакансий
for vacancy in recommended_vacancies:
    print(vacancy['name'])
    print(vacancy['alternate_url'])
    print()

Контент менеджер сайта Детского радио
https://hh.ru/vacancy/103928153

Java-разработчик (Мираполис)
https://hh.ru/vacancy/102570168

Senior Java разработчик (Проект СберСпасибо. Платформа)
https://hh.ru/vacancy/96718596

Программист (Преподаватель в КиберШколе)
https://hh.ru/vacancy/103904832

Java Разработчик
https://hh.ru/vacancy/102615080

Java/Kotlin разработчик
https://hh.ru/vacancy/102575955

Senior Java Developer
https://hh.ru/vacancy/102075546

Java backend developer
https://hh.ru/vacancy/103437017

Middle java developer (Залоги Юридических лиц)
https://hh.ru/vacancy/103364978

Senior QA Engineer
https://hh.ru/vacancy/102693418



# Вывод:

Проект успешно демонстрирует возможность автоматизированного сбора и анализа данных о вакансиях с hh.ru, а также построение прототипа рекомендательной системы, который может помочь пользователям находить подходящие вакансии на основе их навыков.

Парсинг вакансий: Реализован скрипт, который собирает информацию о вакансиях по четырем специальностям, извлекает основную информацию и сохраняет её в удобном для анализа формате.
Анализ данных: Проведен анализ средней зарплаты по каждой специальности и выявлены 10 самых частых биграмм в названиях вакансий.
Агрегация данных: Сравнены два набора данных с вакансиями интернет-маркетологов, что позволило выявить вакансии, присутствующие в одном наборе, но отсутствующие в другом.
Рекомендательная система: Разработан прототип, который на основе введенных ключевых навыков пользователя рекомендует 10 наиболее подходящих вакансий.

Этот проект может служить основой для более сложных и функциональных систем анализа и рекомендаций вакансий, что значительно упростит процесс поиска работы для соискателей.